In [6]:
from neo4j import GraphDatabase
import pickle
import pandas as pd
from geopy.distance import geodesic

# Visualization packages
import networkx as nx
import matplotlib.pyplot as plt
import pygraphviz
import holoviews as hv
from holoviews import dim, opts
hv.extension('bokeh', 'matplotlib')
import hvplot.networkx as hvnx
from networkx.drawing.nx_agraph import graphviz_layout
import geopandas
import contextily as ctx
from shapely.geometry import Point
import geoviews as gv

pd.set_option('display.max_columns', None)

ModuleNotFoundError: No module named 'pygraphviz'

In [ ]:
# MSA dataframe represents the nodes
# attributes: msa, main_city, population, gdp, tti, lat, lon
msa_pickle = '../data/pickled/msa_df.pickle'
with open(msa_pickle, 'rb') as file:
    msa_df = pickle.load(file)
msa_df

In [ ]:
# # Route CONNECTIVITY relationship
# # attributes: drive_distance, drive_duration, flight_frequency, flight_distance, flight_duration_ramp, flight_duration_air, 
# #             flight_full_duration, passenger_volume, capacity, carriers
         
# # Driving distance based on averages from Google Maps
# dist_pickle = '../data/pickled/distance_df.pickle'
# with open(dist_pickle, 'rb') as file:
#     dist_df = pickle.load(file)

# # Create dataframe to look at city pairs
# city_pair_drive = dist_df.copy()
# city_pair_drive['CityPair'] = city_pair_drive.apply(lambda x: tuple(sorted([x['Origin'], x['Destination']])), axis=1)
# city_pair_drive = city_pair_drive.drop(columns=['Origin','Destination'])
# # Groupby
# city_pair_drive = city_pair_drive.groupby('CityPair').mean().reset_index()
# city_pair_drive['City1'] = city_pair_drive['CityPair'].apply(lambda x: x[0])
# city_pair_drive['City2'] = city_pair_drive['CityPair'].apply(lambda x: x[1])
# city_pair_drive = city_pair_drive[['City1','City2','Distance_miles','Duration_minutes']].rename(columns={'Distance_miles':'drive_distance', 'Duration_minutes':'drive_duration'})

# flight_pickle = '../data/pickled/flights_df.pickle'
# with open(flight_pickle, 'rb') as file:
#     flights_df = pickle.load(file)

# flights_df['City1'] = flights_df['CityPair'].apply(lambda x: x[0])
# flights_df['City2'] = flights_df['CityPair'].apply(lambda x: x[1])

# city_pair_flight = flights_df.groupby(['City1','City2']).agg(
#     num_passengers=('PASSENGERS','sum'),
#     total_seats=('SEATS','sum'),
#     flight_frequency=('DEPARTURES_PERFORMED', 'sum'),
#     scheduled=('DEPARTURES_SCHEDULED','sum'),
#     flight_distance=('DISTANCE','mean'),
#     num_carriers=('UNIQUE_CARRIER','nunique'),
#     flight_duration_ramp=('RAMP_TO_RAMP','sum'),
#     flight_duration_air=('AIR_TIME','sum')
# ).reset_index()

# city_pair_flight['flight_duration_ramp'] = city_pair_flight['flight_duration_ramp']/city_pair_flight['flight_frequency']
# city_pair_flight['flight_duration_air'] = city_pair_flight['flight_duration_air']/city_pair_flight['flight_frequency']
# city_pair_flight['flight_duration_total'] = city_pair_flight['flight_duration_ramp'] + 200

# city_pair = pd.merge(city_pair_drive,city_pair_flight, on=['City1', 'City2'], how='outer')
# city_pair = city_pair[['City1', 'City2','drive_distance','drive_duration','flight_distance','flight_duration_ramp','flight_duration_air',
#               'flight_duration_total','num_passengers','total_seats','flight_frequency','scheduled','num_carriers']]

# # Get the "as the crow flies" distance for flight distance if we don't have flight data
# def get_acf_dist(row,msa_df):
#     if pd.isnull(row['flight_distance']):
#         c1 = msa_df.loc[msa_df['MainCity']==row['City1'], ['lat','lng']]
#         c1 = (c1['lat'].values[0],c1['lng'].values[0])
#         c2 = msa_df.loc[msa_df['MainCity']==row['City2'], ['lat','lng']]
#         c2 = (c2['lat'].values[0],c2['lng'].values[0])
#         dist = geodesic(c1,c2).miles
#         return dist
#     return row['flight_distance']

# # Apply the function to calculate the distance for null values
# city_pair['flight_distance'] = city_pair.apply(lambda row: get_acf_dist(row,msa_df), axis=1)

# # Drop Hawaii as it is not a viable option for HSR and fill all other nulls with 0
# city_pair = city_pair[city_pair['drive_distance'].notnull()]
# city_pair.fillna(0,inplace=True)

# # Save the DataFrame as a pickle file
# city_pair_pickle = '../data/pickled/city_pair_df.pickle'
# with open(city_pair_pickle, 'wb') as file:
#     pickle.dump(city_pair, file)

city_pair_pickle = '../data/pickled/city_pair_df.pickle'
with open(city_pair_pickle, 'rb') as file:
    city_pair = pickle.load(file)

### Normalize variables
#### Use min-max normalization

In [ ]:
msa_df['PopNorm'] = (msa_df['Population'] - msa_df['Population'].min()) / (msa_df['Population'].max() - msa_df['Population'].min())
msa_df['GDPNorm'] = (msa_df['GDP'] - msa_df['GDP'].min()) / (msa_df['GDP'].max() - msa_df['GDP'].min())
msa_df['TTINorm'] = (msa_df['TravelTimeIndexValue'] - msa_df['TravelTimeIndexValue'].min()) / (msa_df['TravelTimeIndexValue'].max() - msa_df['TravelTimeIndexValue'].min())

In [ ]:
for cn in city_pair.columns:
    if 'City' in cn:
        continue
    norm = cn+'_norm'
    city_pair[norm] = (city_pair[cn] - city_pair[cn].min()) / (city_pair[cn].max() - city_pair[cn].min())

In [ ]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, data=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query,data))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [ ]:
URI = "bolt://localhost:7687"
USERNAME = "neo4j"
PASSWORD = "HSRProject"
conn = Neo4jConnection(URI, USERNAME, PASSWORD)

conn.query("CREATE OR REPLACE DATABASE hsrdb",{})

In [ ]:
# add MSAs as nodes
msa_dict = msa_df.to_dict(orient='records')

query = '''
    UNWIND $rows AS row
    MERGE (m:MSA {name: row.MainCity})
    SET m.msa = row.MetroArea, 
        m.population = row.Population, 
        m.pop_norm = row.PopNorm,
        m.gdp = row.GDP,
        m.gdp_norm = row.GDPNorm,
        m.tti = row.TravelTimeIndexValue,
        m.tti_norm = row.TTINorm,
        m.lon = row.lng, 
        m.lat = row.lat
    RETURN count(*) as total
    '''
conn.query(query, data={"rows": msa_dict}, db='hsrdb')

In [ ]:
# add city_pairs as edges
cp_dict = city_pair.to_dict(orient='records')

query = '''
    UNWIND $rows as row
    MATCH (m1:MSA {name: row.City1})
    MATCH (m2:MSA {name: row.City2})
    MERGE (m1)<-[:CONNECTIVITY {
        drive_distance: row.drive_distance, 
        drive_duration: row.drive_duration,
        flight_distance: row.flight_distance,
        flight_duration_ramp: row.flight_duration_ramp,
        flight_duration_air: row.flight_duration_air,
        flight_duration_total: row.flight_duration_total,
        num_passengers: row.num_passengers,
        total_seats: row.total_seats,
        flight_frequency: row.flight_frequency,
        scheduled: row.scheduled,
        num_carriers: row.num_carriers,
        drive_distance_norm: row.drive_distance_norm, 
        drive_duration_norm: row.drive_duration_norm,
        flight_distance_norm: row.flight_distance_norm,
        flight_duration_ramp_norm: row.flight_duration_ramp_norm,
        flight_duration_air_norm: row.flight_duration_air_norm,
        flight_duration_total_norm: row.flight_duration_total_norm,
        num_passengers_norm: row.num_passengers_norm,
        total_seats_norm: row.total_seats_norm,
        flight_frequency_norm: row.flight_frequency_norm,
        scheduled_norm: row.scheduled_norm,
        num_carriers_norm: row.num_carriers_norm}]->(m2)
   '''
conn.query(query, data={"rows": cp_dict}, db='hsrdb')

## Initial Graph

In [ ]:
nodes_query = '''
    MATCH (n:MSA)
    RETURN n.name AS name, n.lat AS lat, n.lon AS lon
'''
nodes_results = conn.query(nodes_query, db='hsrdb')
nodes_df = pd.DataFrame([dict(record) for record in nodes_results])

# It is super big to draw all connections, since is is a fully connected/complete graph (all nodes in the network are connected to all other nodes)
# Filter to view only relationships that include flight data
rels_query = """
    MATCH (n)-[r]->(m)
    WHERE r.num_passengers > 0
    RETURN n, r, m
"""
rels_results = conn.query(rels_query, db='hsrdb')

In [ ]:
# Initialize an undirected graph
G = nx.Graph()

# Add nodes from nodes_results
for record in nodes_results:
    G.add_node(record['name'], lat=record['lat'], lon=record['lon'])

# Add edges from relationships_results
for record in rels_results:
    node1 = record['n']
    node2 = record['m']
    rel = record['r']
    G.add_edge(node1['name'], node2['name'], relationship=rel.type)

# Calculate node degrees and determine sizes
degrees = dict(G.degree())
max_degree = max(degrees.values()) if degrees else 1  # Avoid division by zero
nodes_df['degree'] = nodes_df['name'].map(degrees)
nodes_df['size'] = nodes_df['degree'].apply(lambda x: 5 + (x / max_degree) * 15)  # Scale size between 5 and 20

# Create a GeoDataFrame for nodes
gdf = geopandas.GeoDataFrame(nodes_df, geometry=[Point(xy) for xy in zip(nodes_df['lon'], nodes_df['lat'])])

# Create GeoViews points
points = gv.Points(gdf, vdims=['name','size'])

In [ ]:
# Draw the graph using geocoords
# Plot using Holoviews
plot = points.opts(
    opts.Points(
        color='#008080', 
        size='size',  # Size of the points
        tools=['hover'],  # Enable hover tool
        width=800,
        height=600,
        title="MSA Graph",
        line_width=0.5
    )
)

# Plot edges using Holoviews
edges = []
for u, v in G.edges():
    line = [(G.nodes[u]['lon'], G.nodes[u]['lat']), (G.nodes[v]['lon'], G.nodes[v]['lat'])]
    edges.append(line)

edge_lines = gv.Path(edges)

# Combine plots
final_plot = edge_lines.opts(color='gray', line_width=0.5) * plot

# Display the plot
hv.output(final_plot)

In [ ]:
# Draw the graph
pos = graphviz_layout(G)
node_sizes = [(5 + G.degree(node)) * 1 for node in G.nodes()]
node_colors = ['#008080'] * len(G.nodes())
plot_size=(800, 800)

pos_df = pd.DataFrame.from_dict(pos, orient='index', columns=['x', 'y'])
pos_df['index'] = pos_df.index

plot = hvnx.draw(G, pos=pos, node_size=node_sizes, node_color=node_colors, edge_color="gray", edge_line_width=1)
plot.opts(width=plot_size[0], height=plot_size[1])

#labels = hv.Labels(pos_df, ['x', 'y'], 'index')
#plot = plot * labels.opts(xoffset=0, yoffset=-5)

plot

# ANALYSIS

In [ ]:
# Look at travel demand
td_query = '''
    MATCH (m1:MSA)-[r:CONNECTIVITY]->(m2:MSA)
    RETURN m1.name AS City1, m2.name AS City2,
           r.num_passengers AS Passengers, r.total_seats AS Seats,
           r.flight_frequency AS Frequency, r.num_carriers AS Airlines
    ORDER BY Passengers DESC
    LIMIT 10
'''

td_result = conn.query(td_query, db='hsrdb')
td_df = pd.DataFrame([dict(record) for record in td_result])
td_df

In [ ]:
# Maximize passenger volume and minimize distance
eff_query = '''
    MATCH (m1:MSA)-[r:CONNECTIVITY]->(m2:MSA)
    WHERE r.drive_distance_norm > 0
    RETURN m1.name AS City1,
           m2.name AS City2,
           r.num_passengers AS PassengerVolume,
           r.drive_distance AS DriveDistance,
           (r.num_passengers_norm / r.drive_distance_norm) AS EfficiencyScore
    ORDER BY EfficiencyScore DESC
'''

eff_result = conn.query(eff_query, db='hsrdb')
eff_df = pd.DataFrame([dict(record) for record in eff_result])

# write dataframe to csv
eff_df.to_csv('efficiencyScore.csv',index=False)

eff_df.head(10)

In [ ]:
city_pair_weights_df = pd.merge(city_pair, eff_df, on=['City1','City2'], how='left').drop(columns=['PassengerVolume','DriveDistance']).rename(columns={'EfficiencyScore':'efficiency_score'})
city_pair_weights_df.fillna(0,inplace=True)

## Degree Centrality
#### Centrality can help identify key hubs that could maximize network connectivity if connected by HSR.

#### Measures the number of connections (edges) a node has. In a weighted context, it can be interpreted as the sum of the weights of all edges connected to the node.  

#### When weights are used, such as num_passengers, the score reflects the total volume of these weights associated with a city. It effectively captures the overall travel demand involving that city.

#### Create a custom weight to use by combining features:
*demand_weight = num_passengers * 0.8 + flight_frequency * 0.2*

In [ ]:
# Create a demand weight using num_passengers and flight freq
create_dw_query = '''
    MATCH (m1:MSA)-[r:CONNECTIVITY]->(m2:MSA)
    SET r.demand_weight = (r.num_passengers_norm * 0.7) + (r.flight_frequency_norm * 0.3)
    RETURN m1.name AS City1,
           m2.name AS City2,
           r.demand_weight AS DemandWeight
    ORDER BY DemandWeight DESC
'''

dw_result = conn.query(create_dw_query, db='hsrdb')
dw_df = pd.DataFrame([dict(record) for record in dw_result])
dw_df

In [ ]:
# Look at MSAs with high degree centrailty score
demand_proj_query = '''
    CALL gds.graph.project(
        'demandGraph',
        'MSA',
        {
            CONNECTIVITY: {
                orientation: 'UNDIRECTED',
                properties: ['demand_weight']
            }
        }
    )
    YIELD
    graphName AS graph,
    relationshipProjection AS knowsProjection,
    nodeCount AS nodes,
    relationshipCount AS rels
'''

conn.query(demand_proj_query, db='hsrdb')

# Calculate degree centrality using custom weight
demand_deg_query = '''
    CALL gds.degree.stream('demandGraph', { relationshipWeightProperty: 'demand_weight' })
    YIELD nodeId, score
    RETURN gds.util.asNode(nodeId).name AS MSA, score AS DemandScore
    ORDER BY score DESC
'''

demand_deg_result = conn.query(demand_deg_query, db='hsrdb')
demand_deg_df = pd.DataFrame([dict(record) for record in demand_deg_result])
demand_deg_df

In [ ]:
demand_deg_df.head(10)

In [ ]:
# Add demand calculations to data frames
city_pair_weights_df = pd.merge(city_pair_weights_df, dw_df, on=['City1', 'City2'], how='left').rename(columns={'DemandWeight':'demand_weight'})
msa_weights_df = pd.merge(msa_df,demand_deg_df, left_on='MainCity', right_on='MSA', how='left').rename(columns={'DemandScore':'demand_score'})
msa_weights_df.drop(columns=['MSA'], inplace=True)

## Closeness Centrality
#### Closeness centrality measures how quickly a node can reach all other nodes in the network. It is defined as the inverse of the sum of the shortest path distances from the node to all other nodes in the graph. Nodes with a high closeness score have the shortest distances to all other nodes.

#### Can help identify cities that are centrally located in terms of travel time or distance, making them potentially good candidates for hubs.

#### ** Could be really useful if we first cluster cities and then determine closeness centrality.
#### First calculate overall score, then also calculate clustered score

#### Create custom closeness measure using weights
Since the default closeness centrality doesn’t consider weights, you can implement a custom measure to approximate proximity by considering the weighted paths between cities.

Inverse Distance: Use the inverse of the travel distance as a measure of proximity, giving higher scores to cities that are closer together.

In [ ]:
create_distw_query = '''
    MATCH (m1:MSA)-[r:CONNECTIVITY]->(m2:MSA)
    WITH m1.name AS City1, m2.name AS City2, 
         CASE 
           WHEN r.drive_distance > 0 THEN 1.0 / r.drive_distance
           ELSE 0
         END AS proximity_score
    
    // Aggregate the proximity scores for City1
    WITH City1, proximity_score
    RETURN City1 AS City, sum(proximity_score) AS TotalProximityScore
    
    UNION ALL
    
    // Aggregate the proximity scores for City2
    MATCH (m1:MSA)-[r:CONNECTIVITY]->(m2:MSA)
    WITH m1.name AS City1, m2.name AS City2, 
         CASE 
           WHEN r.drive_distance > 0 THEN 1.0 / r.drive_distance
           ELSE 0
         END AS proximity_score
    
    WITH City2, proximity_score
    RETURN City2 AS City, sum(proximity_score) AS TotalProximityScore
'''
distw_result = conn.query(create_distw_query, db='hsrdb')
distw_df = pd.DataFrame([dict(record) for record in distw_result])

final_scores = distw_df.groupby('City').sum().reset_index()
final_scores = final_scores.sort_values(by='TotalProximityScore', ascending=False)
final_scores

In [ ]:
msa_weights_df = pd.merge(msa_weights_df,final_scores, left_on='MainCity', right_on='City', how='left').rename(columns={'TotalProximityScore':'proximity_score'})
msa_weights_df.drop(columns=['City'], inplace=True)

In [ ]:
msa_weights_df

## Community Detection/Clustering
#### Find clusters of cities with strong connectivity to identify potential regions for HSR. 

### Approach 1: 
#### Simply cluster nodes using lat/lon (k-means)

In [ ]:
# Put lat/lon in single field
coord_query = '''
    MATCH (m:MSA)
    SET m.coordinates = [m.lat, m.lon]
'''
conn.query(coord_query, db='hsrdb')

geo_proj = '''                
    MATCH (m1:MSA)-[r:CONNECTIVITY]->(m2:MSA)
    WITH gds.graph.project(
        'geoGraph',
        m1,
        m2,
        {
            sourceNodeProperties: m1 {coordinates: m1.coordinates},
            targetNodeProperties: m2 {coordinates: m2.coordinates}
        }
            
    ) AS g
    RETURN g.graphName, g.nodeCount, g.relationshipCount
'''
conn.query(geo_proj, db='hsrdb')

In [ ]:
kmeans_query = '''
    CALL gds.kmeans.stream('geoGraph', {
        nodeProperty: 'coordinates',
        k: 6
    })
    YIELD nodeId, communityId
    RETURN gds.util.asNode(nodeId).name AS name, gds.util.asNode(nodeId).lat AS lat, gds.util.asNode(nodeId).lon AS lon, communityId
    ORDER BY communityId, name ASC
'''
kmeans_result = conn.query(kmeans_query, db='hsrdb')
kmeans_df = pd.DataFrame([dict(record) for record in kmeans_result])
kmeans_df

In [ ]:
# Initialize an undirected graph
G = nx.Graph()

# Add nodes from nodes_results
for record in kmeans_result:
    G.add_node(record['name'], pos=(record['lon'], record['lat']), cluster=record['communityId'])

# Convert to GeoDataFrame
gdf = geopandas.GeoDataFrame(kmeans_df, geometry=[Point(xy) for xy in zip(kmeans_df['lon'], kmeans_df['lat'])])

# Convert the GeoDataFrame to Holoviews Elements
points = gv.Points(gdf, vdims=['name', 'communityId'])

palette = ['red', 'blue', 'green', 'orange', 'purple','pink']
          
# Visualize using Holoviews
plot = points.opts(
    opts.Points(
        color='communityId',  # Use ClusterId for coloring
        cmap=palette,  # Define the color map
        size=8,  # Size of the points
        tools=['hover'],  # Enable hover tool
        width=800,
        height=600,
        title="City Clusters by ClusterId"
    )
)

# Display the plot
plot

### Approach 2:
#### Perform community detection using drive and flight distance (Leiden). To do this, we use Gaussian weighting for distance based on our optimal range.

In [ ]:
create_gaus_weights = '''
    // Define parameters
    WITH 300.0 AS optimalDistance, 25.0 AS sigma

    // Calculate Gaussian weights for both drive and flight distances
    MATCH (m1:MSA)-[r:CONNECTIVITY]->(m2:MSA)
    WITH m1, m2, r, 
        exp(-((r.drive_distance - optimalDistance) * (r.drive_distance - optimalDistance)) / (2 * sigma * sigma)) AS driveWeight,
        exp(-((r.flight_distance - optimalDistance) * (r.flight_distance - optimalDistance)) / (2 * sigma * sigma))AS flightWeight
    SET r.dist_weight = (driveWeight*0.5) + (flightWeight*0.5)
    RETURN m1.name AS City1, m2.name AS City2, r.drive_distance AS DriveDistance, r.flight_distance AS FlightDistance, r.dist_weight AS Weight
    ORDER BY Weight desc
'''

gaus_dist_result = conn.query(create_gaus_weights, db='hsrdb')
gaus_dist_df = pd.DataFrame([dict(record) for record in gaus_dist_result])
# write dataframe to csv
gaus_dist_df.to_csv('gaussianWeights.csv',index=False)
gaus_dist_df

In [ ]:
city_pair_weights_df = pd.merge(city_pair_weights_df, gaus_dist_df, on=['City1', 'City2'], how='left').rename(columns={'Weight':'gaus_dist_weight'})
city_pair_weights_df.drop(columns=['DriveDistance','FlightDistance'], inplace=True)

In [ ]:
# Use only the connected component
conn_comp = '''
    MATCH (m1:MSA)-[r:CONNECTIVITY]->(m2:MSA)
    WITH gds.graph.project(
        'connectedMSAGraph',
        m1,
        m2,
        { relationshipProperties: r { .dist_weight }},
        { undirectedRelationshipTypes: ['*']}
    ) AS g
    RETURN
        g.graphName AS graph, g.nodeCount AS nodes, g.relationshipCount AS rels
'''
conn.query(conn_comp, db='hsrdb')

In [ ]:
# Leiden community detection algorithm
leiden_query = '''
    CALL gds.leiden.stream('connectedMSAGraph',{ relationshipWeightProperty: 'dist_weight' })
    YIELD nodeId, communityId
    RETURN gds.util.asNode(nodeId).name AS name, gds.util.asNode(nodeId).lat AS lat, gds.util.asNode(nodeId).lon AS lon, communityId
    ORDER BY communityId, name
'''

leiden_result = conn.query(leiden_query, db='hsrdb')
leiden_df = pd.DataFrame([dict(record) for record in leiden_result])
leiden_df

In [ ]:
community_mapping = {old_id: new_id for new_id, old_id in enumerate(sorted(leiden_df['communityId'].unique()))}
# Apply the mapping to the DataFrame
leiden_df['mappedCommunityId'] = leiden_df['communityId'].map(community_mapping)

In [ ]:
# Initialize an undirected graph
G = nx.Graph()

# Add nodes from nodes_results
for record in leiden_result:
    G.add_node(record['name'], pos=(record['lon'], record['lat']), cluster=record['communityId'])

# Convert to GeoDataFrame
gdf = geopandas.GeoDataFrame(leiden_df, geometry=[Point(xy) for xy in zip(leiden_df['lon'], leiden_df['lat'])])

# Convert the GeoDataFrame to Holoviews Elements
points = gv.Points(gdf, vdims=['name', 'mappedCommunityId'])

palette = ['red', 'blue', 'green', 'orange', 'purple','pink']

# Visualize using Holoviews
plot = points.opts(
    opts.Points(
        color='mappedCommunityId',  # Use ClusterId for coloring
        cmap=palette,  # Define the color map
        size=8,  # Size of the points
        tools=['hover'],  # Enable hover tool
        width=800,
        height=600,
        title="City Clusters by Leiden Algorithm"
    )
)

# Display the plot
plot

In [ ]:
leiden_df.drop(columns=['lat','lon'], inplace=True)
msa_weights_df = pd.merge(msa_weights_df,leiden_df, left_on='MainCity', right_on='name', how='left').rename(columns={'mappedCommunityId':'cluster_id'})
msa_weights_df.drop(columns=['name','communityId'], inplace=True)

In [ ]:
msa_weights_df

In [ ]:
# Save the DataFrames as a pickle file
city_pair_weights_pickle = '../data/pickled/city_pair_weights_df.pickle'
with open(city_pair_weights_pickle, 'wb') as file:
    pickle.dump(city_pair_weights_df, file)

msa_weights_pickle = '../data/pickled/msa_weights_df.pickle'
with open(msa_weights_pickle, 'wb') as file:
    pickle.dump(msa_weights_df, file)